In [1]:
import os
import sys
from helpers.logger import LoggerHelper, logging
from helpers.config import ConfigHelper
from helpers import config
from helpers import logger
import numpy as np
import pandas as pd
from classes import preprocessing_functions
from classes.multiclass_models import NeuralNetwork, ConvolutionalNeuralNetwork, RecurrentNeuralNetwork, LongShortTermMemory
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from modules.lib_IF import *
from modules.lib import *
from modules.optimizer import *
from xai_client import Client
import json


In [18]:
import joblib
with open(models['Ideko_model']['pdp_ale_surrogate_model'], 'rb') as f:
                        surrogate_model = joblib.load(f)

In [6]:
indicator_list = ["f3"]

X, Y, Z = preprocessing_functions.read_data('data_subset', indicator_list)


2024-07-15 12:26:06,132 - classes.preprocessing_functions - INFO - read_data(): Read data from path data_subset
2024-07-15 12:26:06,140 - classes.preprocessing_functions - INFO - read_data(): 1 variables are read: ['f3']
2024-07-15 12:26:27,855 - classes.preprocessing_functions - INFO - read_data(): Number of files read 648


In [7]:
X_pad = preprocessing_functions.add_padding(X, indicator_list)

Y_encoded = preprocessing_functions.encode_response_variable(Y)



2024-07-15 12:26:27,880 - classes.preprocessing_functions - INFO - add_padding(): Matching the length of the time series adding padding
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [8]:
X_train, X_test, y_train, y_test,z_train,z_test = preprocessing_functions.split_data(X_pad, Y_encoded, Z)

n_timestamps = X_train.shape[1]
n_features = X_train.shape[2]

n_classes = y_train.shape[1]


In [5]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from keras.layers import Masking, Dense, Flatten, Conv1D, MaxPooling1D, SimpleRNN, LSTM
from keras import Input

def create_model(n_timestamps, n_features, activation_function='relu', units=(50, 50,50), n_classes=2):
    model = Sequential()
    # Input layer
    model.add(Input(shape=(n_timestamps, n_features)))
    # Masking layer
    model.add(Masking(mask_value=0))
    # Fully connected layers
    for unit in units:
        model.add(Dense(units=unit, activation=activation_function))
    # Flatten layer
    model.add(Flatten())
    # Output layer
    model.add(Dense(n_classes, activation="softmax"))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
model = KerasClassifier(build_fn=create_model, 
                        n_timestamps=n_timestamps, 
                        n_features=n_features, 
                        n_classes=n_classes, 
                        verbose=0)

# Define the parameter grid
param_grid = {
    'model__units': [[25, 25, 25],[50, 50, 50],[100, 100, 100]],     # Example values
    'model__activation_function': ['relu', 'tanh'],       # Example values
    'batch_size': (16,32,64) ,                        # Example values
    'epochs': (15,30,50),                       # Example values
}

# Initialize GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)
grid_result = grid.fit(X_train, y_train)

2024-07-12 19:39:51,394 - tensorflow - WARNING - 5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000020F885FD5A0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2024-07-12 19:39:52,117 - tensorflow - WARNING - 5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000020F885FD5A0> triggered tf.function retracing. Tracing is expensive

In [7]:
import dill as pickle
with open('metadata/proxy_data_models/Ideko_nn_grid.pkl', 'wb') as file:
    pickle.dump(grid_result, file)

In [17]:
from xai_client import Client

client = Client()
k = client.get_explanations(explanation_type='hyperparameterExplanation',explanation_method='pdp',model='Ideko_model',feature1='batch_size')